In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [10]:
data = pd.read_json("realdonaldtrump.ndjson", lines=True)

In [12]:
data['created_at']

0       2009-05-04 18:54:25+00:00
1       2009-05-05 01:00:10+00:00
2       2009-05-08 13:38:08+00:00
3       2009-05-08 20:40:15+00:00
4       2009-05-12 14:07:28+00:00
                   ...           
40236   2019-07-11 15:08:23+00:00
40237   2019-07-11 15:52:18+00:00
40238   2019-07-11 15:52:18+00:00
40239   2019-07-11 15:52:19+00:00
40240   2019-07-11 15:52:19+00:00
Name: created_at, Length: 40241, dtype: datetime64[ns, UTC]

In [35]:
target_data = data[(data['created_at']>="2016-04-01") & (data['created_at']<"2016-10-01")]

In [40]:
target_data = [['favorite_count','created_at']]

,favorite_count,created_at
28773,14164,2016-04-01 02:21:02+00:00
28774,14353,2016-04-01 04:23:25+00:00
28775,13177,2016-04-01 04:29:49+00:00
28776,17235,2016-04-01 14:24:46+00:00
28777,30707,2016-04-01 14:25:17+00:00
...,...,...
30703,47434,2016-09-30 18:24:21+00:00
30704,111093,2016-09-30 18:37:48+00:00
30705,11588,2016-09-30 20:03:41+00:00
30706,18015,2016-09-30 20:10:54+00:00


In [60]:
target_data['dayofweek'] = target_data['created_at'].dt.dayofweek+1

C:\Users\18959\AppData\Local\Temp\ipykernel_8260\3529084161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_data['dayofweek'] = target_data['created_at'].dt.dayofweek+1


In [64]:
target_data[['created_at','dayofweek','favorite_count']].groupby('dayofweek').mean()

,favorite_count
dayofweek,
1,16743.804511
2,17879.478261
3,16929.486577
4,18630.339844
5,18099.458333
6,18969.380392
7,20232.597345


In [66]:
ls

 Volume in drive C is OS
 Volume Serial Number is 7AF7-4D88

 Directory of C:\Users\18959\OneDrive - The University of Tokyo\python\sklearn\01_coding\01_coding

2022/06/06  09:30    <DIR>          .
2022/06/06  09:30    <DIR>          ..
2022/06/06  09:30         6,950,560 kindle_review_sample.csv
2022/06/06  09:30       115,901,693 realdonaldtrump.ndjson
               2 File(s)    122,852,253 bytes
               2 Dir(s)  793,095,716,864 bytes free


# kindle

In [ ]:
1. 2012/07/01-2012/12/31の間でReviewは全部でいくつ存在しますか。
2. 2012/07/01-2012/12/31の間で最も多くのreviewを受けた書籍(asin)は何ですか
3. 2012/07/01-2012/12/31の間で最も多くのreviewを受けた書籍(asin)の平均ratingはいくつになりますか。
4. 2012/07/01-2012/12/31の間で最もhelpfulだと評価されたreviewを書いたreviewerIDは何ですか。

In [312]:
kindle = pd.read_csv('kindle_review_sample.csv',sep=',',error_bad_lines=False)

C:\Users\18959\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6281: expected 12 fields, saw 13\n'


In [313]:
kindle.columns

Index(['index', 'Unnamed: 0', 'Unnamed: 0.1', 'asin', 'helpful', 'rating',
       'reviewText', 'reviewTime', 'reviewerID', 'reviewerName', 'summary',
       'unixReviewTime'],
      dtype='object')

In [316]:
kindle['reviewTime'] = pd.to_datetime(kindle['reviewTime'])

In [319]:
kindle[(kindle['reviewTime']>="2012-7-1") & (kindle['reviewTime']<"2013-01-01")].sort_values('reviewTime')

,index,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
5335,4606,4606,2454,B001H55R8M,[2; 3],5,I was pleasantly surprised by the real-world p...,2012-07-01,A1Z9QZ4PJOU61P,Johnny Appleseed,Not your typical; syrupy romance...,1.341101e+09
8448,2665,2665,38677,B004GKMPQW,[11; 13],1,This is the story of a detective and his new p...,2012-07-01,A30YDY4XKVUBUY,Christine Staeven,Author has good pemise - execution really lacks,1.341101e+09
9291,4005,4005,9335,B0030BF28G,[0; 0],4,No one could possibly take this book seriously...,2012-07-01,A2HHKV9CQ03ZRA,Diana M. Hockley,Lighthearted and harmless fun,1.341101e+09
398,3116,3116,1919,B001BXNQ2O,[0; 0],5,This was a good twist; on a already great stor...,2012-07-01,A1U28SQHYF2MST,Anya Alsobrook,OMG... So good!,1.341101e+09
6507,5997,5997,5967,B002HJV4DE,[1; 2],3,..ahkay; I don't mind erotic storys..really I ...,2012-07-02,AL4D6OHMS2Z9P,"M. Rutherford ""Tbascobuzz""",Gimme a break!!,1.341187e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
3586,7568,7568,4961,B002D48NBO,[1; 1],5,"This author turns out his ""Murders"" stories wi...",2012-12-31,A20N2U8KGIX2CM,Winslow,Senior Citizens Are Heroes Too,1.356912e+09
1247,2939,2939,4867,B002D48NBO,[4; 6],2,I got this as a free download; and usually sc...,2012-12-31,A14R9XMZVJ6INB,amf0001,disappointing,1.356912e+09
3117,4522,4522,7351,B002RHP4YQ,[1; 1],4,The story is good; but I got a little confused...,2012-12-31,A1QNGJ3EJTLR6Y,Chelsea Lovely Neese,Heavenly Bride is a good story.,1.356912e+09
5698,1377,1377,9584,B0030CMM4W,[0; 0],1,If you like period pieces you might like this ...,2012-12-31,A3P29QHVZ6JWV8,D Tyler,Don't waste your money.,1.356912e+09


In [320]:
target = kindle[(kindle['reviewTime']>="2012-7-1") & (kindle['reviewTime']<"2013-01-01")].sort_values('reviewTime')

In [326]:
target

,index,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
5335,4606,4606,2454,B001H55R8M,[2; 3],5,I was pleasantly surprised by the real-world p...,2012-07-01,A1Z9QZ4PJOU61P,Johnny Appleseed,Not your typical; syrupy romance...,1.341101e+09
8448,2665,2665,38677,B004GKMPQW,[11; 13],1,This is the story of a detective and his new p...,2012-07-01,A30YDY4XKVUBUY,Christine Staeven,Author has good pemise - execution really lacks,1.341101e+09
9291,4005,4005,9335,B0030BF28G,[0; 0],4,No one could possibly take this book seriously...,2012-07-01,A2HHKV9CQ03ZRA,Diana M. Hockley,Lighthearted and harmless fun,1.341101e+09
398,3116,3116,1919,B001BXNQ2O,[0; 0],5,This was a good twist; on a already great stor...,2012-07-01,A1U28SQHYF2MST,Anya Alsobrook,OMG... So good!,1.341101e+09
6507,5997,5997,5967,B002HJV4DE,[1; 2],3,..ahkay; I don't mind erotic storys..really I ...,2012-07-02,AL4D6OHMS2Z9P,"M. Rutherford ""Tbascobuzz""",Gimme a break!!,1.341187e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
3586,7568,7568,4961,B002D48NBO,[1; 1],5,"This author turns out his ""Murders"" stories wi...",2012-12-31,A20N2U8KGIX2CM,Winslow,Senior Citizens Are Heroes Too,1.356912e+09
1247,2939,2939,4867,B002D48NBO,[4; 6],2,I got this as a free download; and usually sc...,2012-12-31,A14R9XMZVJ6INB,amf0001,disappointing,1.356912e+09
3117,4522,4522,7351,B002RHP4YQ,[1; 1],4,The story is good; but I got a little confused...,2012-12-31,A1QNGJ3EJTLR6Y,Chelsea Lovely Neese,Heavenly Bride is a good story.,1.356912e+09
5698,1377,1377,9584,B0030CMM4W,[0; 0],1,If you like period pieces you might like this ...,2012-12-31,A3P29QHVZ6JWV8,D Tyler,Don't waste your money.,1.356912e+09


In [333]:
target.groupby('asin').sum().sort_values('index')

,index,Unnamed: 0,Unnamed: 0.1,rating,unixReviewTime
asin,,,,,
B001A06VJ8,4,4,1776,4,1.356912e+09
B002BNL3EQ,126,126,4697,5,1.346890e+09
B000OI0FGC,161,161,500,5,1.341619e+09
B004H4XH2I,258,258,39641,1,1.350778e+09
B003370JHG,325,325,11355,3,1.351210e+09
...,...,...,...,...,...
B002EZZJSM,59519,59519,41910,29,1.080095e+10
B002I1XQ1Y,90836,90836,86368,51,1.897335e+10
B002HE1IBW,98181,98181,98528,68,2.296305e+10


In [337]:
target.query("asin=='B002SVQD0A'").mean()

C:\Users\18959\AppData\Local\Temp\ipykernel_8260\1637804854.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  target.query("asin=='B002SVQD0A'").mean()
C:\Users\18959\AppData\Local\Temp\ipykernel_8260\1637804854.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  target.query("asin=='B002SVQD0A'").mean()


index             5.744846e+03
Unnamed: 0        5.744846e+03
Unnamed: 0.1      7.713692e+03
rating            3.500000e+00
unixReviewTime    1.349568e+09
dtype: float64

In [341]:
target.sort_values('helpful',ascending=False)

,index,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
2915,2202,2202,12478,B0035N1V7K,[9; 13],2,Well where should I start ..... I finished rea...,2012-11-15,A3NVB6HKMUH2NM,"Douglas C. Meeks ""Book Reviews @ Large""",Well If You Can't Seduce Them I Guess It's OK ...,1.352938e+09
4237,7192,7192,11056,B0032UPUOQ,[9; 12],2,I love Lora Leigh and have read most of her bo...,2012-11-21,A1HH85PGO4AUKC,"Sam Brown ""romance book lover""",Needs a safe word!!!!,1.353456e+09
7867,6523,6523,28959,B0044UHVBS,[8; 9],2,I was expecting to like this book; but just no...,2012-11-08,A18V52M2SGMKXE,Nell Daley,Just could not get into this book,1.352333e+09
4871,1270,1270,764,B000U0NSV8,[8; 8],3,I read this as I enjoyed the first one of thes...,2012-09-15,A741547755BZ2,"Harley ""Harley""",Not as good as the first one.,1.347667e+09
1106,134,134,9786,B0030IM7KK,[8; 8],4,If you are quite like me and love all things t...,2012-09-11,A3M5F7W63UL56L,Lalla Rookh,a worthy addition,1.347322e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
9047,11707,11707,6583,B002M3I34I,[0; 0],4,I found this series surprisingly addictive.Muc...,2012-10-29,A3L7FD7KC9TOM5,"Amazon Customer ""em""",Love this Series,1.351469e+09
9152,2173,2173,5768,B002HE1IBW,[0; 0],4,WOW; lots of fire works in this story. Great t...,2012-10-29,A2Y0YGKPIYJ5NC,Amazon Customer,Men in Uniforms=HOT; men out of their uniforms...,1.351469e+09
2427,6572,6572,6591,B002M3SXQG,[0; 0],4,Great tails that i can see being told before a...,2012-10-29,A1GA0R83RLIPDW,"Amazon Customer ""MAT""",Great Tails,1.351469e+09
2727,10301,10301,9040,B002Z13UO0,[0; 0],4,I enjoyed the first book in this new series. ...,2012-10-29,A60G200VVF399,Sandala,Great new series - but needs an editor,1.351469e+09
